In [40]:
import cv2
import os
from enum import Enum
from scipy.signal import wiener
from skimage.restoration import unsupervised_wiener, richardson_lucy 

In [41]:
# Noise & Artifact Removal
# Resize images
# Possibly change all views to a single view

In [42]:
# Flags
class NoiseRemoval(Enum):
    GAUSSIAN = 1
    MEDIAN = 2
    BILATERAL = 3
    NONE = 4

class Resize(Enum):
    MIN = 1
    MAX = 2
    NONE = 3

class Color(Enum):
    GRAYSCALE = 1
    RGB = 2

class BlurRemoval(Enum):
    WIENER_FILTER = 1
    BLIND_DECONVOLUTION = 2
    RICHARDSON_LUCY = 3
    NONE = 4

class View(Enum): # TODO: Figure out what to do with views
    SINGLE = 1
    ALL = 2

In [43]:
# Constants
DATASET_PATH = 'dataset'
FLOODED_PATH = 'dataset/flooded'
NON_FLOODED_PATH = 'dataset/non-flooded'

In [44]:
path = os.getcwd()

images_paths = []
images = []
labels = []

folder_path = os.path.join(path, FLOODED_PATH)
for image in os.listdir(folder_path):
    image_path = os.path.join(folder_path, image)
    images_paths.append(image_path)
    images.append(cv2.imread(image_path))
    labels.append(1)

folder_path = os.path.join(path, NON_FLOODED_PATH)
for image in os.listdir(folder_path):
    image_path = os.path.join(folder_path, image)
    images_paths.append(image_path)
    images.append(cv2.imread(image_path))
    labels.append(0)

print('Dataset size: ', len(images))
print('Flooded images: ', labels.count(1))
print('Non-flooded images: ', labels.count(0))

Dataset size:  922
Flooded images:  461
Non-flooded images:  461


In [45]:
# Noise Removal Techniques
# TODO: Add more noise removal techniques
# TODO: Check if the parameters need to be changed
def noise_removal(image, noise_removal):
    if noise_removal == NoiseRemoval.GAUSSIAN:
        return cv2.GaussianBlur(image, (5, 5), 0)
    elif noise_removal == NoiseRemoval.MEDIAN:
        return cv2.medianBlur(image, 5)
    elif noise_removal == NoiseRemoval.BILATERAL:
        return cv2.bilateralFilter(image, 9, 75, 75)
    else:
        return image

In [46]:
# Blur Removal Techniques
def blur_removal(image, blur_removal):
    if blur_removal == BlurRemoval.WIENER_FILTER:
        return wiener(image, (5, 5))
    elif blur_removal == BlurRemoval.BLIND_DECONVOLUTION:
        return unsupervised_wiener(image)
    elif blur_removal == BlurRemoval.RICHARDSON_LUCY:
        return richardson_lucy(image, 5)
    else:
        return image

In [47]:
def get_resize_dims(images):
    max_resize_dims = None
    min_resize_dims = None

    # Iterate over all the images to find the maximum or minimum dimensions
    min_h, min_w = float("inf"), float("inf")
    for image in images:
        h, w, _ = image.shape
        min_h = min(min_h, h)
        min_w = min(min_w, w)
    min_dimension = min(min_h, min_w)
    min_resize_dims = (min_dimension, min_dimension)

    max_h, max_w = 0, 0
    for image in images:
        h, w, _ = image.shape
        max_h = max(max_h, h)
        max_w = max(max_w, w)
    max_dimension = max(max_h, max_w)
    max_resize_dims = (max_dimension, max_dimension)

    return min_resize_dims, max_resize_dims

In [48]:
min_resize_dims, max_resize_dims = get_resize_dims(images)

In [49]:
print('Minimum resize dimensions: ', min_resize_dims)
print('Maximum resize dimensions: ', max_resize_dims)

Minimum resize dimensions:  (123, 123)
Maximum resize dimensions:  (4720, 4720)


In [50]:
# Resize images
def resize_image(image, resize):
    resize_dims = min_resize_dims if resize == Resize.MIN else max_resize_dims
    return cv2.resize(image, resize_dims, interpolation=cv2.INTER_AREA)

In [51]:
# Main pipeline
def preprocess(image, noise=NoiseRemoval.GAUSSIAN, blur=BlurRemoval.WIENER_FILTER, resize=Resize.MIN, color=Color.GRAYSCALE):
    if color == Color.GRAYSCALE:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    image = resize_image(image, resize)

    image = noise_removal(image, noise)
    
    image = blur_removal(image, blur)

    return image

In [52]:
# Pipeline for all images
for i in range(len(images)):
    images[i] = preprocess(images[i])

c:\Users\Coformatique\Abdullah\GitHub\Flooding-Detection\.venv\lib\site-packages\scipy\signal\_signaltools.py:1629: RuntimeWarning: divide by zero encountered in divide
  res *= (1 - noise / lVar)
c:\Users\Coformatique\Abdullah\GitHub\Flooding-Detection\.venv\lib\site-packages\scipy\signal\_signaltools.py:1629: RuntimeWarning: invalid value encountered in multiply
  res *= (1 - noise / lVar)
